In [1]:
import json
import pandas as pd
from sklearn import preprocessing
import numpy as np
import cv2

from torchvision.io import read_image
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim


import matplotlib.path as mplpath

import torch.utils.data as data_utils

import matplotlib.patches as patches
from pathlib import Path


In [2]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
le = preprocessing.LabelEncoder()

In [3]:
def data(file_dir_path):
    
    """
    iterates a json-list of image info and returns format
    [ [ [bounding box], is_chinese, symbol, height, width], ... ]
    
    structure of nested lists are: 
    list of images = [[[ [bb], symbol, height, width    ], [bb, symbol ]], [], []]
    
    """
    with open(file_dir_path) as j_file:
        list_of_marks = [] 
        json_list = list(j_file)
        p = 0 
        final = {}
        for json_str in json_list:
            if p > 1000:
                return final, list_of_marks
            img = json.loads(json_str)
            print(img)
            image_list = []
            for key, value in img.items():
                bounding_box_list = []

                img_id = img['image_id']+'.jpg'
                for v in img['annotations']: #all annotations per image

                    for dic in v:
                        coords = dic['polygon']
                        #coords = dic['adjusted_bbox']
                        isch = dic['is_chinese']
                        text = dic['text']
                        list_of_marks.append(text)

                        
                        bounding_box_list.append([coords[0],coords[1],coords[2],coords[3],isch, str(text)])
                    p += 1
            final[img_id] = bounding_box_list
                    
         

            
    final = [final]
    return final, list_of_marks

In [4]:
#print(ext_bb_from_dir('a1/train.jsonl'))
poly_box = data('a1/train.jsonl')

{'annotations': [[{'adjusted_bbox': [140.26028096262758, 897.1957001682758, 22.167573140645146, 38.36424196832945], 'attributes': ['distorted', 'raised'], 'is_chinese': True, 'polygon': [[140.26028096262758, 896.7550603352049], [162.42785410327272, 898.0769798344178], [162.42785410327272, 935.7929346470926], [140.26028096262758, 935.0939571156308]], 'text': '明'}, {'adjusted_bbox': [162.42785410327272, 898.5416545674744, 23.376713493771263, 37.74268246537315], 'attributes': ['distorted', 'raised'], 'is_chinese': True, 'polygon': [[162.42785410327272, 898.0769798344178], [185.80456759704398, 899.4710040335876], [185.80456759704398, 936.5300382257251], [162.42785410327272, 935.7929346470926]], 'text': '海'}, {'adjusted_bbox': [185.80456759704398, 899.90363223333, 21.764526356269812, 37.08391855844286], 'attributes': ['distorted', 'raised'], 'is_chinese': True, 'polygon': [[185.80456759704398, 899.4710040335876], [207.5690939533138, 900.7688886328147], [207.5690939533138, 937.2163070747966]

In [5]:
#print(poly_box)
marks = poly_box[1]
le.fit_transform(marks)

array([127, 152,  70, ..., 151,  39,   8])

In [6]:
def make_box(box_dict):
    tp = dict()
    #tp_bool = []
    tp_box = []
    X = []
    y = []
    for key, value in box_dict.items():
        print('New Image for processing ... ')
        img = cv2.imread('images/'+key)


        #normalize

        print(img.shape)
        grid1 = np.array( [[[a,b] for b in list(range(img.shape[0]))] for a in list(range(img.shape[1]))]) 
        grid1.shape = (grid1.shape[0]*grid1.shape[1], 2)
        
        img.shape = (3 ,2048, 2048)
        img = np.resize(img, (1, 2048//2, 2048//2))
#       img = img/255
        
        # resize img
        for boxes in value:
            x1 = boxes[0]

            x2 = boxes[1]
            x3 = boxes[2]
            x4 = boxes[3]
            
            #if x1 > 2048:
             #   continue
            #if x2 > 2048:
             #   continue
            #if x3 > 2048:
            #    continue
            #if x4 > 2048:
             #   continue
            
            #badd work around for out-of-image pixels in bounding boxes.
            #Polygon:
            if any(i > 2048 for i in x1 ):
                continue
            
            if any(i > 2048 for i in x2 ):
                continue
                
            if any(i > 2048 for i in x3 ):
                continue
                
            if any(i > 2048 for i in x4 ):
                continue
        
            is_bool = boxes[4]
            mark = boxes[5]
            mark = le.transform([mark])
#            print(mark)
            
            
            """
            begin by resizing img and points. Try multiplyign/dividing by the same number.
            
            This is so the Y-production will smoothen and reduce in time. 
            
            also permute img to (3, s1, s2)
            
                       
            """
            
            
            grid2 = grid1
            
           # pnts = np.array([ x1 , x2, x3, x4 ], dtype=np.int32 )           
            #works

            pnts = np.array([ [x1[0],x1[1] ] , [ x2[0], x2[1] ] , [ x3[0],x3[1] ], [x4[0] ,x4[1] ] ], dtype=np.int32 )           
#            grid1 = np.array( [[[a,b] for b in list(range(img.shape[0]))] for a in list(range(img.shape[1]))]) 
 #           grid1.shape = (grid1.shape[0]*grid1.shape[1], 2)
            path = mplpath.Path(pnts)     
            new_new = path.contains_points(grid2)
            new_new = np.array(new_new, int)
            new_new = np.resize(new_new, (1, 2048//2, 2048//2))
           # new_new = new_new / 255
            new_new = torch.as_tensor(new_new, dtype=torch.float32)
            img = torch.as_tensor(img, dtype=torch.float32)
            tp_box.append((img, new_new))
           # tp_box.append((torch.tensor(img, dtype=torch.long), torch.tensor(new_new, dtype=torch.long)))
            #X.append(torch.LongTensor(img))
            #y.append(torch.LongTensor(new_new))
            
            



            
            #Choose Mark, specific
            # tp_box.append([new_new, mark]) 
        
        break
    return tp_box
    #return X, y


In [7]:
mkb = make_box(poly_box[0])




New Image for processing ... 
(2048, 2048, 3)


In [120]:
for i in mkb:
    inputs, labels = i
    #labels = torch.reshape(labels, (1, 1024*1024))
    print(labels.shape)
#    labels = labels.long()
#    inputs = inputs.long()


torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024, 1024])
torch.Size([1, 1024,

In [110]:


dataset = mkb

In [111]:
#print(dataset[0])

#Make Train, test, val
from sklearn.model_selection import train_test_split

test_size = int(len(dataset)*0.2)

val_size = int(len(dataset)*0.1)
train_size = len(dataset) - (test_size+val_size)

train_d = dataset[:train_size] # 07
val_d = dataset[train_size:-test_size] #01
test_d = dataset[(train_size+val_size):] # 02 


trainloader = torch.utils.data.DataLoader(train_d, batch_size=42,
                                          shuffle=True, num_workers=2)



In [114]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 2, 5)
        self.conv2 = nn.Conv2d(2, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*253*253, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        print(x.shape)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        print(x.shape)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [115]:
net = Net()

In [116]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([2, 1, 5, 5])


In [117]:
#batch_size = 4
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [118]:
for i in trainloader:
    inputs, labels = i
    #inputs.type(torch.LongTensor)
    print(labels.shape)


torch.Size([30, 1, 1024, 1024])


In [119]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        print(torch.max(labels, 1)[0].flatten(1))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
       # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([30, 1, 1024, 1024])
torch.Size([30, 2, 510, 510])
torch.Size([30, 16, 253, 253])
torch.Size([30, 1024144])


RuntimeError: 1D target tensor expected, multi-target not supported